In [ ]:
import sqlite3

# filename to form database
file = "Sqlite3.db"

try:
  conn = sqlite3.connect(file)
  print("Database Sqlite3.db formed.")
except:
  print("Database Sqlite3.db not formed.")

Database Sqlite3.db formed.


In [ ]:
# Importing Sqlite3 Module
import sqlite3

try:
    # Making a connection between sqlite3 database and Python Program
    sqliteConnection = sqlite3.connect('SQLite_Retrieving_data.db')
    # If sqlite3 makes a connection with python program then it will print "Connected to SQLite"
    # Otherwise it will show errors
    print("Connected to SQLite")
except sqlite3.Error as error:
    print("Failed to connect with sqlite3 database", error)
finally:
    # Inside Finally Block, If connection is open, we need to close it
    if sqliteConnection:
        # using close() method, we will close the connection
        sqliteConnection.close()
        # After closing connection object, we will print "the sqlite connection is closed"
        print("the sqlite connection is closed")

Connected to SQLite
the sqlite connection is closed


In [ ]:
myfile='/content/sample_data/Doctorlist1.csv'
import csv
csv.register_dialect('piper', delimiter='|', quoting=csv.QUOTE_NONE)
#Then, use your dialect on the file:

with open(myfile, "r") as csvfile:
    for row in csv.DictReader(csvfile, dialect='piper'):
        print (row.get('Hospital Name'))

None
None
None


In [ ]:
with open(myfile, 'r') as file:
                # Using csv reader with '|' as delimiter
                reader = csv.reader(file, delimiter='|')
                # Skip header row
                next(reader, None)
                for row in reader:
                  print(row[0])


Dr. John Smith
Dr. Sarah Johnson
Dr. Michael Lee


In [ ]:
pip install openai

In [ ]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.4
    Uninstalling openai-1.59.4:
      Successfully uninstalled openai-1.59.4


In [8]:
import openai
from datetime import datetime, timedelta
import json
import sqlite3
import csv

class DoctorAppointmentBot:
    def __init__(self, api_key, db_path='doctor_appointments.db'):
        """Initialize the chatbot with OpenAI API key and database connection."""
        openai.api_key = api_key
        self.db_path = db_path
        self.available_slots = self._generate_sample_slots()
        self.conversation_history = []
        self._initialize_database()

    def _initialize_database(self):
        """Initialize SQLite database and create necessary tables."""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()

                # Create DoctorList table
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS DoctorList (
                        doctor_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT NOT NULL,
                        specialization TEXT NOT NULL,
                        schedule TEXT NOT NULL,
                        max_patients_per_day INTEGER,
                        contact_number TEXT,
                        email TEXT
                    )
                ''')

                # Create Appointments table
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS Appointments (
                        appointment_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        doctor_id INTEGER,
                        patient_name TEXT NOT NULL,
                        appointment_date TEXT NOT NULL,
                        appointment_time TEXT NOT NULL,
                        reason TEXT,
                        status TEXT DEFAULT 'scheduled',
                        FOREIGN KEY (doctor_id) REFERENCES DoctorList(doctor_id)
                    )
                ''')

                conn.commit()
        except sqlite3.Error as e:
            print(f"Database initialization error: {e}")

    def import_doctors_from_file(self, file_path):
        """Import doctor information from a pipe-delimited file."""
        try:
            with open(file_path, 'r') as file:
                # Using csv reader with '|' as delimiter
                reader = csv.reader(file, delimiter='|')
                # Skip header row
                next(reader, None)

                with sqlite3.connect(self.db_path) as conn:
                    cursor = conn.cursor()

                    for row in reader:
                        if len(row) >= 6:  # Ensure we have all required fields
                            cursor.execute('''
                                INSERT INTO DoctorList (
                                    name, specialization, schedule,
                                    max_patients_per_day, contact_number, email
                                ) VALUES (?, ?, ?, ?, ?, ?)
                            ''', (row[0], row[1], row[2], int(row[3]), row[4], row[5]))

                    conn.commit()
                print("Doctor information imported successfully!")

        except FileNotFoundError:
            print(f"Error: File {file_path} not found")
        except sqlite3.Error as e:
            print(f"Database error: {e}")
        except Exception as e:
            print(f"Error importing doctor information: {e}")

    def get_all_doctors(self):
        """Retrieve all doctors from the database."""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute("SELECT * FROM DoctorList")
                return cursor.fetchall()
        except sqlite3.Error as e:
            print(f"Error retrieving doctors: {e}")
            return []

    def _generate_sample_slots(self):
        """Generate sample available appointment slots for next 7 days."""
        slots = {}
        start_date = datetime.now()
        for i in range(7):
            current_date = start_date + timedelta(days=i)
            date_str = current_date.strftime("%Y-%m-%d")
            slots[date_str] = [
                "09:00 AM", "10:00 AM", "11:00 AM",
                "02:00 PM", "03:00 PM", "04:00 PM"
            ]
        return slots


    def _get_chatgpt_response(self, user_input):

      #Get response from ChatGPT API with database validation.

      # First, get available doctors and slots from database
      available_doctors = self._get_available_doctors()
      available_slots = self._get_available_slots()
      self.conversation_history.append({"role": "user", "content": user_input})

      # Format the doctor and slot information for the system prompt
      doctor_info = "\n".join([
          f"- Dr. {doc['name']}: {doc['specialization']} (ID: {doc['id']}, Schedule: {doc['schedule']})"
          for doc in available_doctors
      ])

      # Create a comprehensive system prompt with database information
      system_prompt = f"""
      You are a medical appointment scheduling assistant with access to the following information:

      Available Doctors:
      {doctor_info}

      Available Time Slots:
      {json.dumps(available_slots, indent=2)}

      Your tasks:
      1. Validate user requests against available doctors and slots
      2. Collect required information:
        - Patient name
        - Reason for visit
        - Preferred doctor (must match available doctors)
        - Preferred date and time (must match available slots)
        - Contact information

      When scheduling:
      1. Only suggest available doctors and time slots
      2. If a requested doctor/time is unavailable, suggest alternatives
      3. Format appointment requests in JSON when ready to book

      Response format for booking:
      APPOINTMENT_REQUEST{{
          "doctor_id": <id>,
          "date": "YYYY-MM-DD",
          "time": "HH:MM AM/PM",
          "patient_info": {{
              "name": "<name>",
              "reason": "<reason>",
              "contact": "<contact>"
          }}
      }}

      Be professional and HIPAA-compliant.
      """
      # Prepare messages for ChatGPT
      messages = [
          {"role": "system", "content": system_prompt},
          *self.conversation_history
          ]

      try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=500,
                temperature=0.7
            )
            assistant_response = response.choices[0].message.content
            self.conversation_history.append({"role": "assistant", "content": assistant_response})
            return assistant_response

      except Exception as e:
            return f"Error: {str(e)}"

def _get_available_doctors(self):
    """Retrieve available doctors from database."""
    try:
      with sqlite3.connect(self.db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("""
                    SELECT
                        doctor_id,
                        name,
                        specialization,
                        schedule,
                        max_patients_per_day
                    FROM DoctorList
                """)
        doctors = cursor.fetchall()
        return [
                    {
                        'id': doc[0],
                        'name': doc[1],
                        'specialization': doc[2],
                        'schedule': doc[3],
                        'max_patients': doc[4]
                    }
        for doc in doctors
                ]
    except sqlite3.Error as e:
            print(f"Database error: {e}")
            return []

def _get_available_slots(self):
  """Get available slots based on existing appointments."""
  try:
    with sqlite3.connect(self.db_path) as conn:
      cursor = conn.cursor()

      # Get all booked appointments
      cursor.execute("""
                    SELECT
                        doctor_id,
                        appointment_date,
                        appointment_time
                    FROM Appointments
                    WHERE status = 'scheduled'
                """)
      booked_slots = cursor.fetchall()

      # Create availability dictionary
      available_slots = {}
      doctors = self._get_available_doctors()

      # Generate slots for next 7 days
      for doctor in doctors:
            available_slots[doctor['id']] = {}
            # Parse doctor's schedule
            schedule_days = self._parse_schedule(doctor['schedule'])
            # Generate slots for next 7 days
            current_date = datetime.now()
            for i in range(7):
              date = current_date + timedelta(days=i)
              date_str = date.strftime('%Y-%m-%d')

              # Check if doctor works on this day
              if date.strftime('%a') in schedule_days:

              # Generate default time slots
                day_slots = [
                                "09:00 AM", "10:00 AM", "11:00 AM",
                                "02:00 PM", "03:00 PM", "04:00 PM"
                            ]

                # Remove booked slots
                for booking in booked_slots:
                  if (booking[0] == doctor['id'] and
                                    booking[1] == date_str and
                                    booking[2] in day_slots):
                                    day_slots.remove(booking[2])

                  available_slots[doctor['id']][date_str] = day_slots

                return available_slots

  except sqlite3.Error as e:
            print(f"Database error: {e}")
  return {}

    def _parse_schedule(self, schedule):
      """Parse schedule string into list of days."""
      # Convert schedule like "Mon-Wed-Fri" to ["Mon", "Wed", "Fri"]
      return schedule.split('-')

    def _validate_doctor(self, doctor_id):
        """Validate if doctor exists and is available."""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute("""
                    SELECT COUNT(*)
                    FROM DoctorList
                    WHERE doctor_id = ?
                """, (doctor_id,))
                return cursor.fetchone()[0] > 0
        except sqlite3.Error:
            return False

    def _validate_time_slot(self, doctor_id, date_str, time_str):
        """Validate if the requested time slot is available."""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()

                # Check if slot is already booked
                cursor.execute("""
                    SELECT COUNT(*)
                    FROM Appointments
                    WHERE doctor_id = ?
                    AND appointment_date = ?
                    AND appointment_time = ?
                    AND status = 'scheduled'
                """, (doctor_id, date_str, time_str))

                if cursor.fetchone()[0] > 0:
                    return False

                # Check if doctor works on this day
                cursor.execute("""
                    SELECT schedule
                    FROM DoctorList
                    WHERE doctor_id = ?
                """, (doctor_id,))

                schedule = cursor.fetchone()
                if not schedule:
                    return False

                # Parse the date string to get day of week
                appointment_date = datetime.strptime(date_str, '%Y-%m-%d')
                day_of_week = appointment_date.strftime('%a')

                # Check if day is in doctor's schedule
                return day_of_week in schedule[0].split('-')

        except sqlite3.Error:
            return False

    def book_appointment(self, doctor_id, date_str, time_str, patient_info):
      """Book an appointment in the database."""
      try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()

                # Check if slot is available
                cursor.execute('''
                    SELECT COUNT(*) FROM Appointments
                    WHERE doctor_id = ? AND appointment_date = ? AND appointment_time = ?
                    AND status = 'scheduled'
                ''', (doctor_id, date_str, time_str))

                if cursor.fetchone()[0] > 0:
                    return {
                        "status": "error",
                        "message": "Selected slot is already booked"
                    }

                # Book the appointment
                cursor.execute('''
                    INSERT INTO Appointments (
                        doctor_id, patient_name, appointment_date,
                        appointment_time, reason, status
                    ) VALUES (?, ?, ?, ?, ?, ?)
                ''', (doctor_id, patient_info['name'], date_str, time_str,
                      patient_info['reason'], 'scheduled'))

                conn.commit()

                return {
                    "status": "success",
                    "message": f"Appointment booked for {date_str} at {time_str}",
                    "patient_info": patient_info
                }

      except sqlite3.Error as e:
            return {
                "status": "error",
                "message": f"Database error: {str(e)}"
            }

  def process_user_input(self, user_input):
    """Process user input and return appropriate response."""
    ai_response = self._get_chatgpt_response(user_input)
      # If response contains appointment request, validate it
    if "APPOINTMENT_REQUEST" in ai_response:
      try:
                  # Extract appointment details
                  request_str = ai_response.split("APPOINTMENT_REQUEST")[1]
                  request_data = json.loads(request_str)

                  # Validate doctor
                  if not self._validate_doctor(request_data['doctor_id']):
                    return "I apologize, but the selected doctor is not available. Would you like to see a list of available doctors?"

                  # Validate time slot
                  if not self._validate_time_slot(
                      request_data['doctor_id'],
                      request_data['date'],
                      request_data['time']
                  ):
                      return "I apologize, but the selected time slot is not available. Would you like to see alternative time slots?"

                  # If validation passes, keep the original response
                  self.conversation_history.append({"role": "assistant", "content": ai_response})
                  return ai_response

      except json.JSONDecodeError:
                  return "I apologize, but there was an error processing the appointment request. Could you please provide the details again?"

      if "APPOINTMENT_REQUEST" in ai_response:
            try:
                request_details = json.loads(ai_response.split("APPOINTMENT_REQUEST")[1])
                doctor_id = request_details.get("doctor_id")
                date_str = request_details["date"]
                time_str = request_details["time"]
                patient_info = request_details["patient_info"]

                booking_result = self.book_appointment(doctor_id, date_str, time_str, patient_info)
                return booking_result["message"]
            except json.JSONDecodeError:
                return ai_response

      return ai_response

def main():

      # Initialize the chatbot with your OpenAI API key
      api_key = ""
      bot = DoctorAppointmentBot(api_key)

      # Import doctors from file
      bot.import_doctors_from_file('/content/sample_data/Doctorlist.csv')

      print("Welcome to the Doctor Appointment Chatbot!")
      print("Type 'quit' to exit")
      print("\nAvailable doctors:")
      doctors = bot.get_all_doctors()
      for doctor in doctors:
        print(f"ID: {doctor[0]}, Name: {doctor[1]}, Specialization: {doctor[2]}")

        while True:

          user_input = input("\nYou: ")
          if user_input.lower() == 'quit':
                break
          response = bot.process_user_input(user_input)
          print(f"Bot: {response}")

if __name__ == "__main__":
        main()

Doctor information imported successfully!
Welcome to the Doctor Appointment Chatbot!
Type 'quit' to exit

Available doctors:
ID: 1, Name: Dr. John Smith, Specialization: Cardiology

You: book appointment with smith


AttributeError: 'DoctorAppointmentBot' object has no attribute 'process_user_input'